##### Imports

In [1]:
#Adapted from   /mnt/data/supratik/demonstrateDIR/v2_evalDIRParamDataGeneration.py
import argparse
from argparse import Namespace
import os
import re
from functools import reduce
from glob import glob
from time import sleep
import json
import csv
import nibabel as nib

import imageio #Bose: Imageio is a Python library that provides an easy interface to read and write a wide range of image data, including animated images, volumetric data
import matplotlib.pyplot as plt
import matplotlib.cm as cm #Bose: matplotlib colormaps and functions
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize #Bose: The matplotlib.colors module is used for converting color or numbers arguments to RGBA or RGB.This module is used for mapping numbers to colors or color specification conversion in a 1-D array of colors also known as colormap.And Normalize class is used to normalize data into the interval of [0.0, 1.0].
from skimage.segmentation import mark_boundaries #Bose: Return image with boundaries between labeled regions highlighted
from skimage.transform import rescale #Bose: Rescale operation resizes an image by a given scaling factor. The scaling factor can either be a single floating point value, or multiple values - one along each axis.
colormap = cm.hsv
norm = Normalize()

import pydicom
from scipy.ndimage import morphology

from torch.nn import MSELoss
from viu.io import volume
from viu.io.volume import read_volume
from viu.torch.deformation.fields import DVF, set_identity_mapping_cache
from viu.torch.io.deformation import *
from viu.util.body_mask import seg_body
from viu.util.memory import fmt_mem_size
from viu.util.config import json_config
from viu.torch.visualization.ortho_utils import save_ortho_views #from pamomo.visualization.ortho_utils import save_ortho_views
from viu.torch.measure.voi import measure_voi_list

from pamomo.pca.cmo_pca import CMoPCA
from pamomo.registration.deformable import reg, force_unload
from pamomo.visualization.cmo_pca_plots import *
from pamomo.metrices.residual_deformation import *

In [2]:
from exampleUtils import *
mse = MSELoss()

##### Helper function

In [3]:
#Move to exampleUtils
# def saveOriginalAndCoronalTiff(vols, hdfFilePath):
#     # https://forum.image.sc/t/saving-an-nd-image-with-pyimagej/52497/2
#     # tifffile directly or via scikit-image works great for such cases. Here you first have to re-arrange your 
#     # dimensions to ZCYX order and then use tifffile.imwrite:
#     import tifffile
#     tiffFilePath = hdfFilePath.replace('.hdf', '.tiff')
#     tifffile.imwrite(tiffFilePath, np.moveaxis(vols, 0,1), imagej=True, metadata={'axes': 'ZCYX'})
#     tiffFilePath_coronal = hdfFilePath.replace('.hdf', '_coronal.tiff')
#     tifffile.imwrite(tiffFilePath_coronal, np.flip(vols.transpose((3,0,1,2)),axis=2), imagej=True, metadata={'axes': 'ZCYX'})


In [4]:
#Move to exampleUtils
# def read_pca_write_ycomp(pca_fn,  prefilter):
#     import tifffile
#     pca = CMoPCA(example_msk=vol_idx_msk,prefilter=prefilter)
#     pca_mean, pca_res, pca_pos = pca.read(pca_fn, device='cuda')
#     print(f'pca_mean type {pca_mean.type} shape {pca_mean.shape} dtype {pca_mean.dtype} device {pca_mean.device}')
#     print(f'pca_res shape {pca_res.shape} dtype {pca_res.dtype} ')
#     print(f'pca_pos shape {pca_pos.shape} dtype {pca_pos.dtype} ')
#     pca_u=pca.u
#     print(f'pca_u type {pca_u.type} shape {pca_u.shape} dtype {pca_u.dtype} device {pca_u.device}')
#     pca_s=pca.s
#     print(f'pca_s type {pca_s.type} shape {pca_s.shape} dtype {pca_s.dtype} device {pca_s.device}')
#     pca_vt=pca.vt
#     print(f'pca_vt type {pca_vt.type} shape {pca_vt.shape} dtype {pca_vt.dtype} device {pca_vt.device}')
#     pca_all_comp_channel_y = pca_u[:,:,:,:,1].cpu().numpy()
#     print(f'pca_all_comp_channel_y shape {pca_all_comp_channel_y.shape} dtype {pca_all_comp_channel_y.dtype} ')
#     pca_all_comp_channel_y_coronal_tiffFilePath = pca_fn.replace('.hdf', '_pca_all_comp_channel_y_coronal.tiff')
#     tifffile.imwrite(pca_all_comp_channel_y_coronal_tiffFilePath, np.flip(pca_all_comp_channel_y.transpose((3,0,1,2)),axis=2), imagej=True, metadata={'axes': 'ZCYX'})

##### Choose patient

In [4]:
# workingFolderParent = '/home/wd974888/Downloads'
tmpWorkingFolder = '/home/wd974888/Downloads/tmpPCAResults'
patiendId = 7 #in range(8,9): #range(1,12) #range(1,12) #Use <truncateDepth_initial, final> : Pat01:<75, 50>;  Pat04:<0, 0>; Pat07:<95, 95>; Pat09:<200, 100>;
binningType = 'PB' # in ['AB']: #['AB', 'PB']
patientMRN = f'Patient{patiendId:02d}{binningType}'
print(f'patientMRN {patientMRN}')
truncateDepth_initial=95
truncateDepth_final=95
currentDirOptionsKey = f"config01" #in [f"config{i:02d}" for i in range(10,11)]: #[f"config{i:02d}" for i in range(1,11)]
print(f'currentDirOptionsKey {currentDirOptionsKey}')
tiffNotSaved=False
print(f'tiffNotSaved {tiffNotSaved}')

patientMRN Patient07PB
currentDirOptionsKey config01
tiffNotSaved False


##### Get all path

In [6]:
#Get all the paths

# org_vols_hdf_path = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/{patientMRN}_test_vols.hdf'
org_vols_hdf_path = f'{tmpWorkingFolder}/{patientMRN}/{patientMRN}_test_vols.hdf'
print(f'org_vols_hdf_path {org_vols_hdf_path} exists:{os.path.exists(org_vols_hdf_path)}')

# cache_fn =  f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Cache/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.pt'
cache_fn =  f'{tmpWorkingFolder}/{patientMRN}/Cache/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.pt'
print(f'Interpol hardcoded cache file name: {cache_fn} exists:{os.path.exists(cache_fn)}')

# functional_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
functional_pca_fn = f'{tmpWorkingFolder}/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'functional_pca_fn: {functional_pca_fn} exists:{os.path.exists(functional_pca_fn)}')

# functional_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
functional_reconstructedVol_fn = f'{tmpWorkingFolder}/{patientMRN}/Functional_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'functional_reconstructedVol_fn: {functional_reconstructedVol_fn} exists:{os.path.exists(functional_reconstructedVol_fn)}')

# intFuncMixed_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
intFuncMixed_pca_fn = f'{tmpWorkingFolder}/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'intFuncMixed_pca_fn: {intFuncMixed_pca_fn} exists:{os.path.exists(intFuncMixed_pca_fn)}')

# intFuncMixed_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
intFuncMixed_reconstructedVol_fn = f'{tmpWorkingFolder}/{patientMRN}/IntFuncMixed_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'intFuncMixed_reconstructedVol_fn: {intFuncMixed_reconstructedVol_fn} exists:{os.path.exists(intFuncMixed_reconstructedVol_fn)}')

# interpol_pca_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
interpol_pca_fn = f'{tmpWorkingFolder}/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_test_pca.hdf'
print(f'interpol_pca_fn: {interpol_pca_fn} exists:{os.path.exists(interpol_pca_fn)}')

# interpol_reconstructedVol_fn = f'{workingFolderParent}/workingFolder/DeformationExperiment/PCA/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
interpol_reconstructedVol_fn = f'{tmpWorkingFolder}/{patientMRN}/Interpol_{currentDirOptionsKey}_{patientMRN}_reconstructed_vols.hdf'
print(f'interpol_reconstructedVol_fn: {interpol_reconstructedVol_fn} exists:{os.path.exists(interpol_reconstructedVol_fn)}')


org_vols_hdf_path /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Patient07PB_test_vols.hdf exists:True
Interpol hardcoded cache file name: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Cache/Interpol_config01_Patient07PB_test_pca.pt exists:True
functional_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Functional_config01_Patient07PB_test_pca.hdf exists:True
functional_reconstructedVol_fn: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Functional_config01_Patient07PB_reconstructed_vols.hdf exists:True
intFuncMixed_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/IntFuncMixed_config01_Patient07PB_test_pca.hdf exists:True
intFuncMixed_reconstructedVol_fn: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/IntFuncMixed_config01_Patient07PB_reconstructed_vols.hdf exists:True
interpol_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Interpol_config01_Patient07PB_test_pca.hdf exists:True
interpol_reconstructedVol_fn: /home/wd974888/Downloads/tmpPCA

##### Read all volumes

In [7]:
print(f"############ Reading org_vols_hdf ##########")
vols_o, res_o, pos_o = volume.read_volume(org_vols_hdf_path , hdf_ds_name='volumes')
print(f'vols_o shape {vols_o.shape} dtype {vols_o.dtype} min {np.min(vols_o)} max {np.max(vols_o)}')
print(f'res_o {res_o}')
print(f'pos_o {pos_o}')

print(f"############ Reading cache_fn ##########")
vols, dvfs, res, pos, dvf_res, dvf_pos, msks, vol_idx_msk = torch.load(cache_fn)
print(f'vols shape {vols.shape} dtype {vols.dtype} device {vols.device}  min {torch.min(vols)} max {torch.max(vols)}') #
print(f'res {res}')
print(f'pos {pos}')
print(f'dvfs shape {dvfs.shape} dtype {dvfs.dtype} device {dvfs.device} min {torch.min(dvfs)} max {torch.max(dvfs)}') #
print(f'dvf_res {dvf_res}')
print(f'dvf_pos {dvf_pos}')

print(f"############ Reading functional_reconstructed_vols ##########")
functional_reconstructed_vols, functional_reconstructed_res, functional_reconstructed_pos = volume.read_volume(functional_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'functional_reconstructed_vols shape {functional_reconstructed_vols.shape} dtype {functional_reconstructed_vols.dtype}   min {np.min(functional_reconstructed_vols)} max {np.max(functional_reconstructed_vols)}') #
print(f'functional_reconstructed_res {functional_reconstructed_res}')
print(f'functional_reconstructed_pos {functional_reconstructed_pos}')
# reconstructed_vols=torch.tensor(reconstructed_vols, device=vols.device)

print(f"############ Reading intFuncMixed_reconstructed_vols ##########")
intFuncMixed_reconstructed_vols, intFuncMixed_reconstructed_res, intFuncMixed_reconstructed_pos = volume.read_volume(intFuncMixed_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'intFuncMixed_reconstructed_vols shape {intFuncMixed_reconstructed_vols.shape} dtype {intFuncMixed_reconstructed_vols.dtype}   min {np.min(intFuncMixed_reconstructed_vols)} max {np.max(intFuncMixed_reconstructed_vols)}') #
print(f'intFuncMixed_reconstructed_res {intFuncMixed_reconstructed_res}')
print(f'intFuncMixed_reconstructed_pos {intFuncMixed_reconstructed_pos}')

print(f"############ Reading interpol_reconstructed_vols ##########")
interpol_reconstructed_vols, interpol_reconstructed_res, interpol_reconstructed_pos = volume.read_volume(interpol_reconstructedVol_fn, hdf_ds_name='volumes')
print(f'interpol_reconstructed_vols shape {interpol_reconstructed_vols.shape} dtype {interpol_reconstructed_vols.dtype}   min {np.min(interpol_reconstructed_vols)} max {np.max(interpol_reconstructed_vols)}') #
print(f'interpol_reconstructed_res {interpol_reconstructed_res}')
print(f'interpol_reconstructed_pos {interpol_reconstructed_pos}')


############ Reading org_vols_hdf ##########
vols_o shape (10, 331, 512, 512) dtype float32 min -1507.0 max 9116.0
res_o [0.9765625 0.9765625 1.       ]
pos_o [ 1.7187500e-03 -1.9849828e+02 -3.8400002e+01]
############ Reading cache_fn ##########
vols shape torch.Size([10, 331, 512, 512]) dtype torch.float32 device cpu  min -1507.0 max 9116.0
res [0.9765625 0.9765625 1.       ]
pos [ 1.71875000e-03 -1.98498281e+02 -3.84000000e+01]
dvfs shape torch.Size([1, 10, 166, 256, 256, 3]) dtype torch.float32 device cpu min DVF(-0.0810) max DVF(0.0618)
dvf_res tensor([1.9570, 1.9570, 2.0000], dtype=torch.float64)
dvf_pos tensor([0., 0., 0.], dtype=torch.float64)
############ Reading functional_reconstructed_vols ##########
functional_reconstructed_vols shape (10, 331, 512, 512) dtype float32   min -1681.77001953125 max 8625.8046875
functional_reconstructed_res [0.9765625 0.9765625 1.       ]
functional_reconstructed_pos [ 1.7187500e-03 -1.9849828e+02 -3.8400002e+01]
############ Reading intFuncMi

##### Display one bin of vols

In [8]:
# Let us display one bin
from exampleUtils import *
display_vol = vols[0,...].cpu().numpy()
v1_volumeComparisonViewer3D(
    listVolumes=[display_vol],listLabels=[f'{patientMRN}_bin00'],
    maxZ0=display_vol.shape[0], maxZ1=display_vol.shape[1], maxZ2=display_vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500, useAspectCol=False)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

##### Save tiff format of volumes in original and in coronal view

In [9]:
if tiffNotSaved:
    print(f"############ Saving tiff of original bins ##########")
    saveOriginalAndCoronalTiff(vols=vols_o, hdfFilePath=org_vols_hdf_path)
    print(f"############ Saving tiff of functional_reconstructed bins ##########")
    saveOriginalAndCoronalTiff(vols=functional_reconstructed_vols, hdfFilePath=functional_reconstructedVol_fn)
    print(f"############ Saving tiff of intFuncMixed_reconstructed bins ##########")
    saveOriginalAndCoronalTiff(vols=intFuncMixed_reconstructed_vols, hdfFilePath=intFuncMixed_reconstructedVol_fn)
    print(f"############ Saving tiff of interpol_reconstructed bins ##########")
    saveOriginalAndCoronalTiff(vols=interpol_reconstructed_vols, hdfFilePath=interpol_reconstructedVol_fn)
else:
    print(f'Tiffs are already saved.')

Tiffs are already saved.


##### Read (few) PCA components computed using functional and interpol warping and save y components as Tiff 

In [10]:
read_pca_write_ycomp(pca_fn=functional_pca_fn, vol_idx_msk=vol_idx_msk, prefilter=False)
read_pca_write_ycomp(pca_fn=intFuncMixed_pca_fn, vol_idx_msk=vol_idx_msk,  prefilter=True)
read_pca_write_ycomp(pca_fn=interpol_pca_fn, vol_idx_msk=vol_idx_msk, prefilter=True)


Read CMoPCA from /home/wd974888/Downloads/tmpPCAResults/Patient07PB/Functional_config01_Patient07PB_test_pca.hdf ...
  mean volume ...


  principal components ...
  sigma and coefficients ...
pca_mean type <built-in method type of Tensor object at 0x7f5c5cb2b5e0> shape torch.Size([331, 512, 512]) dtype torch.float32 device cuda:0
pca_res shape (3,) dtype float32 
pca_pos shape (3,) dtype float32 
pca_u type <built-in method type of DVF object at 0x7f5b8800b9f0> shape torch.Size([9, 331, 512, 512, 3]) dtype torch.float32 device cuda:0
pca_s type <built-in method type of Tensor object at 0x7f5c5cb987c0> shape torch.Size([9]) dtype torch.float32 device cuda:0
pca_vt type <built-in method type of Tensor object at 0x7f5c5cb907c0> shape torch.Size([10, 9]) dtype torch.float32 device cuda:0
pca_all_comp_channel_y shape (9, 331, 512, 512) dtype float32 
Read CMoPCA from /home/wd974888/Downloads/tmpPCAResults/Patient07PB/IntFuncMixed_config01_Patient07PB_test_pca.hdf ...
  mean volume ...
  principal components ...
  sigma and coefficients ...
pca_mean type <built-in method type of Tensor object at 0x7f5c5cbdc400> shape torch.S

In [11]:
# functional_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient09PB/Functional_config01_Patient09PB_test_pca.hdf exists:True
# intFuncMixed_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient09PB/IntFuncMixed_config01_Patient09PB_test_pca.hdf exists:True
# interpol_pca_fn: /home/wd974888/Downloads/tmpPCAResults/Patient09PB/Interpol_config01_Patient09PB_test_pca.hdf exists:True